In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-09 03:53:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-01-09 03:53:31 (1.69 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

1. Filter the date to retrieve all rows where total_votes >= 20

In [ ]:
filtered_vine_df = vine_df.filter('total_votes>=20')
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

2. Create new dataframe to retrieve all rows where number helpful_votes / total_votes >= 50%

In [ ]:
filtered_vine_2_df = filtered_vine_df.withColumn('votes_greater_than_50', filtered_vine_df["helpful_votes"]/filtered_vine_df["total_votes"]).filter('votes_greater_than_50>0.5')
filtered_vine_2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_greater_than_50|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|                  1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|                  1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|                  0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|              0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|   0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|   0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|   0.7727272727272727|


3. Filter the dataframe and create a new one that retrieves all the rows where review was a paid one (vine == "Y")

In [ ]:
paid_reviews_df = filtered_vine_2_df.filter('vine == "Y"')
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_greater_than_50|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|   0.8709677419354839|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|   0.7619047619047619|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|   0.8780487804878049|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|   0.9733333333333334|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|                  1.0|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|                  1.0|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|                 0.95|


4. Create another dataframe with unpaid reviews

In [ ]:
unpaid_reviews_df = filtered_vine_2_df.filter('vine == "N"')
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_greater_than_50|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|                  1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|                  1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|                  0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|              0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|   0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|   0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|   0.7727272727272727|


5. Determine the following for paid and unpaid reviews:

*   Total number of reviews
*   Number of 5-star reviews
*   % of 5-star reviews

In [ ]:
# Total number of paid reviews
total_paid_reviews = paid_reviews_df.count()
print(total_paid_reviews)

333


In [ ]:
# Number of 5 star paid reviews
five_star_paid = paid_reviews_df.filter('star_rating==5').count()
print(five_star_paid)

139


In [ ]:
# Percent of 5-star paid reviews
percent_five_star_paid = five_star_paid/total_paid_reviews
print(percent_five_star_paid)

0.4174174174174174


In [ ]:
# Total number of unpaid reviews
total_unpaid_reviews = unpaid_reviews_df.count()
print(total_unpaid_reviews)

61474


In [ ]:
# Number of 5 star unpaid reviews
five_star_unpaid = unpaid_reviews_df.filter('star_rating==5').count()
print(five_star_unpaid)

32646


In [ ]:
# Percent of 5-star unpaid reviews
percent_five_star_unpaid = five_star_unpaid/total_unpaid_reviews
print(percent_five_star_unpaid)

0.5310537788333279
